#### train process
1. import csv to df
1. process df to be modelable
  1. add variables
    * dow (cat - 7 vars)
    * hr (cat - 24 vars)
        * pair dow with hr?
    * slow_day (cat - 0, 1)
    * fast_day (cat - 0, 1)
    * L2
        * log L2?
    * google_dist (needs id?)
    * weather (needs datetime?)
1. model
1. use cooksd + fox criterion to identify outliers
1. filter outliers and make new df

## * need to drop outliers

## * 아래 2칸 실행

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [3]:
# 경로 변수 (고정)

dataLoc = '../_data/'

trainfile = 'train_03_997.csv'
old_trainfile = 'train.csv'
kaggle_trainfile = 'train_k.csv'

testfile = 'test_clean.csv'
old_testfile = 'test.csv'

train_171016 = '171016_dow_hr_pas_L2_dur_only.csv'
train_osrm = '../_data/train_with_osrm.csv'
train_osrm_weather = '../_data/train_with_osrm_and_weather.csv'
train_osrm_weather_hourly = '../_data/train_with_osrm_and_weather_hourly.csv'
train_dropped_nan_fixed = '../_data/train__dropped__no_disaster_nan_fixed.csv'
train_latest = '../_data/train__dropped__lnglat__saved__no_disaster_nan_fixed.csv'

## `train__dropped__lnglat__saved__no_disaster_nan_fixed` (전처리 전)

In [1]:
pd.options.display.max_columns = 40

In [4]:
# train_with_osrm_and_weather.csv
df_train = pd.read_csv(dataLoc + train_latest)
df_train.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,p_lng,p_lat,d_lng,d_lat,dur,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,-73.987793,40.724792,-73.975616,40.656445,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,-73.957596,40.717770,-73.951424,40.775230,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,-74.000954,40.742031,-73.947708,40.782200,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,-73.985733,40.738258,-73.993179,40.754890,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,-74.006615,40.740650,-73.985619,40.723362,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [5]:
df_train.isnull().sum()

is_hol           0
m_d              0
dow              0
hr               0
L2               0
pas_cnt          0
p_lng            0
p_lat            0
d_lng            0
d_lat            0
dur              0
total_dist       0
total_time       0
day_max_temp     0
day_min_temp     0
day_rainfall     0
day_snowfall     0
day_snowdepth    0
hr_temp          0
hr_humid         0
hr_dewpt         0
hr_windspeed     0
hr_gustspeed     0
hr_precip        0
dtype: int64

## rid outliers

In [6]:
# p_lng is idx2
# ...
# d_lat is idx5

df_copy = df_train.copy()

for i in range(6, 11):
    lower = np.percentile(df_train.iloc[:, i], 0.3)
    upper = np.percentile(df_train.iloc[:, i], 99.7)
    df_copy = df_copy[ (df_copy.iloc[:, i] > lower) \
                         & (df_copy.iloc[:, i] < upper) ]

df_copy.shape
  

(683021, 24)

In [7]:
droppable_new = ['total_time', 'p_lng', 'p_lat', 'd_lng', 'd_lat']

for col in droppable_new:
    try:
        del df_copy[col]
    except:
        continue

In [8]:
df_copy.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,dur,total_dist,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,1454,10010.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,1409,10357.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,1081,9275.8,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,800,2726.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,1151,3455.8,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [12]:
model0 = sm.OLS.from_formula('dur1 ~ C(is_hol) + C(m_d) + C(dow) + C(hr) + C(pas_cnt)\
                            + scale(total_dist) + day_max_temp + day_min_temp + day_rainfall\
                            + day_snowfall + day_snowdepth + hr_temp + hr_humid + hr_dewpt\
                            + hr_windspeed + hr_gustspeed + hr_precip - 1', data=df_copy)
res0 = model0.fit()
res0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   dur1   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     7770.
Date:                Sat, 21 Oct 2017   Prob (F-statistic):               0.00
Time:                        09:57:33   Log-Likelihood:            -4.9592e+06
No. Observations:              683021   AIC:                         9.919e+06
Df Residuals:                  682864   BIC:                         9.920e+06
Df Model:                         156                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
C(is_hol)[False]      548.5046     74.206      7.392      0.000     403.063     693.946
C(is_hol)[True]       479.8965     81.349      5.899      0.000     320.456     639.337
C(m_d)[T.102]         -48.0572      6.267     -7.668      0.000     -60.341     -35.773
C(m_d)[T.103]         -35.6020      5.568     -6.395      0.000     -46.514     -24.690
C(m_d)[T.104]         -16.5902      8.318     -1.995      0.046     -32.892      -0.288
C(m_d)[T.105]          -1.8506      9.130     -0.203      0.839     -19.745      16.044
C(m_d)[T.106]         -24.6205      6.688     -3.681      0.000     -37.729     -11.512
C(m_d)[T.107]         -40.6839      5.405     -7.527      0.000     -51.277     -30.090
C(m_d)[T.108]           0.8853      7.172      0.123      0.902     -13.171      14.942
C(m_d)[T.109]         -33.6940      4.637     -7.266      0.000     -42.783     -24.605
C(m_d)[T.110]         -35.8608      3.566    -10.056      0.000     -42.850     -28.871
C(m_d)[T.111]          26.7495      7.195      3.718      0.000      12.648      40.851
C(m_d)[T.112]         -23.6380      5.138     -4.601      0.000     -33.708     -13.568
C(m_d)[T.113]          31.1839      9.678      3.222      0.001      12.215      50.153
C(m_d)[T.114]           6.6192      7.194      0.920      0.357      -7.480      20.719
C(m_d)[T.115]          13.0987      5.831      2.247      0.025       1.671      24.526
C(m_d)[T.116]         -37.4920      4.229     -8.865      0.000     -45.781     -29.203
C(m_d)[T.117]          28.2426      5.769      4.896      0.000      16.936      39.549
C(m_d)[T.118]          29.6283      6.142      4.824      0.000      17.590      41.666
C(m_d)[T.119]          81.6852      9.365      8.722      0.000      63.329     100.041
C(m_d)[T.120]          34.0887      7.575      4.500      0.000      19.242      48.935
C(m_d)[T.121]          70.7168      7.768      9.104      0.000      55.492      85.942
C(m_d)[T.122]         102.5080     12.062      8.498      0.000      78.866     126.150
C(m_d)[T.123]          -3.4033      0.969     -3.513      0.000      -5.302      -1.505
C(m_d)[T.124]        -134.6091      4.906    -27.436      0.000    -144.225    -124.993
C(m_d)[T.125]          80.5291      4.615     17.450      0.000      71.484      89.574
C(m_d)[T.126]          33.2649      5.750      5.785      0.000      21.995      44.535
C(m_d)[T.127]          44.7938      4.560      9.822      0.000      35.856      53.732
C(m_d)[T.128]          33.2846      5.128      6.490      0.000      23.233      43.336
C(m_d)[T.129]          57.1170      7.185      7.950      0.000      43.035      71.199
C(m_d)[T.130]          18.1281      4.913      3.690      0.000       8.500      27.756
C(m_d)[T.131]         -25.9881      5.248     -4.952      0.000     -36.275     -15.702
C(m

In [33]:
model00 = sm.OLS.from_formula('dur1 ~ C(is_hol) + C(dow) + C(hr)\
                            + scale(total_dist) + scale(day_max_temp) + scale(day_rainfall)\
                            + scale(day_snowfall) + scale(day_snowdepth) + scale(hr_temp) + scale(hr_humid)\
                            + scale(hr_windspeed)', data=df_copy)
res00 = model00.fit()
res00.summary()

KeyboardInterrupt: 

In [18]:
res0.aic

9918616.329233449

In [32]:
sm.stats.anova_lm(res00, typ=2)

,sum_sq,df,F,PR(>F)
C(is_hol),1.072931e+10,2.0,4.482240e+04,0.000000e+00
C(dow),1.529227e+09,6.0,2.129482e+03,0.000000e+00
C(hr),5.527142e+09,23.0,2.007826e+03,0.000000e+00
scale(total_dist),1.385615e+11,1.0,1.157700e+06,0.000000e+00
scale(day_max_temp),4.774688e+06,1.0,3.989315e+01,2.684057e-10
scale(day_rainfall),5.040446e+05,1.0,4.211359e+00,4.015453e-02
scale(day_snowfall),1.263102e+07,1.0,1.055338e+02,9.367930e-25
scale(day_snowdepth),3.501673e+08,1.0,2.925695e+03,0.000000e+00
scale(hr_temp),7.707144e+06,1.0,6.439421e+01,1.020166e-15
scale(hr_humid),4.763717e+05,1.0,3.980149e+00,4.603989e-02


In [53]:
model1 = sm.OLS.from_formula('dur ~ C(is_hol) + C(m_d) + C(dow) + C(hr) + C(pas_cnt)\
                            + total_dist + day_rainfall\
                            + day_snowfall + day_snowdepth - 1', data=df_copy)
res1 = model1.fit()
res1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dur   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     8077.
Date:                Fri, 20 Oct 2017   Prob (F-statistic):               0.00
Time:                        21:52:38   Log-Likelihood:            -4.9593e+06
No. Observations:              683021   AIC:                         9.919e+06
Df Residuals:                  682870   BIC:                         9.921e+06
Df Model:                         150                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             376.7398     91.885      4.100      0.000     196.648     556.832
C(is_hol)[T.True]    -109.1365      5.780    -18.883      0.000    -120.464     -97.809
C(m_d)[T.102]         -88.4769      4.912    -18.013      0.000     -98.104     -78.850
C(m_d)[T.103]         -57.2623      4.994    -11.465      0.000     -67.051     -47.474
C(m_d)[T.104]         -81.1097      4.955    -16.370      0.000     -90.821     -71.398
C(m_d)[T.105]         -87.8354      4.749    -18.494      0.000     -97.144     -78.527
C(m_d)[T.106]         -77.0820      4.652    -16.569      0.000     -86.200     -67.964
C(m_d)[T.107]         -74.9829      4.520    -16.587      0.000     -83.843     -66.123
C(m_d)[T.108]         -23.8753      5.764     -4.142      0.000     -35.172     -12.579
C(m_d)[T.109]         -31.9611      4.279     -7.468      0.000     -40.349     -23.573
C(m_d)[T.110]         -39.6391      3.418    -11.597      0.000     -46.338     -32.940
C(m_d)[T.111]         -25.0011      4.759     -5.253      0.000     -34.329     -15.673
C(m_d)[T.112]         -50.1120      4.519    -11.089      0.000     -58.969     -41.255
C(m_d)[T.113]         -52.3723      4.349    -12.043      0.000     -60.896     -43.849
C(m_d)[T.114]         -51.8138      4.404    -11.764      0.000     -60.446     -43.181
C(m_d)[T.115]           6.7992      5.721      1.189      0.235      -4.413      18.012
C(m_d)[T.116]         -31.9774      4.231     -7.558      0.000     -40.270     -23.685
C(m_d)[T.117]           1.1911      4.505      0.264      0.791      -7.639      10.021
C(m_d)[T.118]         -10.7589      6.752     -1.593      0.111     -23.993       2.476
C(m_d)[T.119]           1.5347      4.501      0.341      0.733      -7.286      10.356
C(m_d)[T.120]         -27.5195      4.457     -6.175      0.000     -36.255     -18.784
C(m_d)[T.121]           5.0801      4.344      1.170      0.242      -3.434      13.594
C(m_d)[T.122]          12.7830      5.668      2.255      0.024       1.674      23.892
C(m_d)[T.123]           2.8334      0.602      4.704      0.000       1.653       4.014
C(m_d)[T.124]        -139.1408      5.004    -27.805      0.000    -148.949    -129.333
C(m_d)[T.125]          72.8444      4.604     15.823      0.000      63.821      81.867
C(m_d)[T.126]          67.8650      4.496     15.095      0.000      59.053      76.677
C(m_d)[T.127]          43.5640      4.557      9.559      0.000      34.632      52.496
C(m_d)[T.128]           5.5429      4.400      1.260      0.208      -3.082      14.168
C(m_d)[T.129]          30.3625      5.518      5.503      0.000      19.548      41.177
C(m_d)[T.130]          -8.9541      4.160     -2.152      0.031     -17.107      -0.801
C(m_d)[T.131]           0.3834      4.594      0.083      0.933      -8.620       9.387
C(m

In [58]:
model2 = sm.OLS.from_formula('dur ~ C(is_hol) + C(dow) + C(hr)\
                            + scale(total_dist) - 1', data=df_copy)
res2 = model2.fit()
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dur   R-squared:                       0.634
Model:                            OLS   Adj. R-squared:                  0.634
Method:                 Least Squares   F-statistic:                 3.818e+04
Date:                Fri, 20 Oct 2017   Prob (F-statistic):               0.00
Time:                        21:55:46   Log-Likelihood:            -4.9644e+06
No. Observations:              683021   AIC:                         9.929e+06
Df Residuals:                  682989   BIC:                         9.929e+06
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
C(is_hol)[False]      728.6464      2.433    299.543      0.000     723.879     733.414
C(is_hol)[True]       605.8267      3.702    163.657      0.000     598.571     613.082
C(dow)[T.Monday]      -70.5258      1.599    -44.098      0.000     -73.660     -67.391
C(dow)[T.Saturday]    -63.5704      1.503    -42.306      0.000     -66.516     -60.625
C(dow)[T.Sunday]     -137.1394      1.585    -86.533      0.000    -140.246    -134.033
C(dow)[T.Thursday]     12.3694      1.522      8.127      0.000       9.386      15.353
C(dow)[T.Tuesday]     -17.3797      1.557    -11.162      0.000     -20.431     -14.328
C(dow)[T.Wednesday]    -4.5554      1.539     -2.960      0.003      -7.572      -1.539
C(hr)[T.1]            -14.6268      3.424     -4.271      0.000     -21.338      -7.915
C(hr)[T.2]            -33.2158      3.766     -8.820      0.000     -40.597     -25.834
C(hr)[T.3]            -52.6253      4.174    -12.608      0.000     -60.806     -44.445
C(hr)[T.4]           -119.1322      4.690    -25.404      0.000    -128.323    -109.941
C(hr)[T.5]           -235.3429      4.815    -48.877      0.000    -244.780    -225.906
C(hr)[T.6]           -156.2081      3.616    -43.200      0.000    -163.295    -149.121
C(hr)[T.7]             11.9699      3.120      3.837      0.000       5.855      18.085
C(hr)[T.8]            139.7001      2.984     46.817      0.000     133.852     145.548
C(hr)[T.9]            165.9576      2.970     55.874      0.000     160.136     171.779
C(hr)[T.10]           166.4158      2.988     55.696      0.000     160.560     172.272
C(hr)[T.11]           180.6602      2.961     61.004      0.000     174.856     186.465
C(hr)[T.12]           181.9785      2.925     62.205      0.000     176.245     187.712
C(hr)[T.13]           173.8093      2.927     59.377      0.000     168.072     179.547
C(hr)[T.14]           191.6765      2.902     66.054      0.000     185.989     197.364
C(hr)[T.15]           204.2609      2.922     69.907      0.000     198.534     209.988
C(hr)[T.16]           190.0664      2.984     63.687      0.000     184.217     195.916
C(hr)[T.17]           187.5792      2.879     65.150      0.000     181.936     193.222
C(hr)[T.18]           161.0308      2.787     57.782      0.000     155.569     166.493
C(hr)[T.19]            99.4103      2.793     35.590      0.000      93.936     104.885
C(hr)[T.20]            45.2328      2.831     15.977      0.000      39.684      50.782
C(hr)[T.21]            26.1328      2.838      9.207      0.000      20.570      31.696
C(hr)[T.22]            30.9005      2.868     10.774      0.000      25.279      36.522
C(hr)[T.23]             8.5284      2.954      2.887      0.004       2.738      14.319
scale(total_dist)     453.6600      0.423   1073.437      0.000     452.832     454.488
===

## cross validation

In [9]:
df_copy['dur1'] = df_copy['dur']
df_copy.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,dur,total_dist,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur1
0,False,430,Saturday,23,0.069424,1,1454,10010.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,430,Saturday,23,0.057791,1,1409,10357.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,430,Saturday,23,0.066698,2,1081,9275.8,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,430,Saturday,23,0.018223,1,800,2726.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,430,Saturday,23,0.027198,1,1151,3455.8,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [10]:
del df_copy['dur']
df_copy.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,total_dist,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur1
0,False,430,Saturday,23,0.069424,1,10010.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,430,Saturday,23,0.057791,1,10357.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,430,Saturday,23,0.066698,2,9275.8,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,430,Saturday,23,0.018223,1,2726.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,430,Saturday,23,0.027198,1,3455.8,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [11]:
df_copy.to_csv('../_data/train__dropped__shaved__cleaned__no_disaster.csv', index=False)

In [64]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


cv = KFold(10)

scores = np.zeros(10)
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(df_copy.iloc[:, :-1], df_copy.iloc[:, -1], test_size=0.33)
    df_train = pd.concat([X_train, y_train], axis=1)
    print("trial {}".format(i+1))
    model_train = sm.OLS.from_formula('dur1 ~ C(is_hol) + C(dow) + C(hr)\
                            + scale(total_dist) - 1', data=df_train)
    res_train = model_train.fit()
    y_pred = res_train.predict(X_test)
    y_pred[y_pred < 0] = 0
    try:
        scores[i] = mean_squared_log_error(y_test, y_pred)
        print(pd.concat([y_test, y_pred], axis=1).head(3))
    except:
        print("except")
        print(y_test[y_test<=0].shape)
        print(y_pred[y_pred<=0].shape)
        break

np.sqrt(scores)

trial 1
        dur1            0
653259   974   783.286058
465847   988   567.151195
671106  1193  1010.496532
trial 2
        dur1           0
410155   939  722.007771
605593   903  516.077693
104165   869  775.878568
trial 3
        dur1           0
20306    233  481.752589
410059   616  711.636286
105695   866  832.742284
trial 4
        dur1            0
385879  1580  1001.633252
349928  1526   817.213673
221774   471   612.218012
trial 5
        dur1           0
699476   330  532.083513
582223   465  666.128535
334971  1006  813.953979
trial 6
        dur1            0
229961   793   873.756387
323520   367   540.329001
205496  3136  3306.212054
trial 7
        dur1            0
573195  2963  1353.671320
222048  1394  1144.828518
577313   430   633.793168
trial 8
        dur1           0
545937   868  596.219877
30983    402  687.666575
671477   251  510.724194
trial 9
        dur1            0
345910  1348  1287.135540
694494   429   589.506712
54531   1664   905.502080
trial 10

array([ 0.48857102,  0.48920087,  0.48979733,  0.49069593,  0.48954785,
        0.49052741,  0.48854601,  0.49091676,  0.48863866,  0.49123123])

In [67]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold



cv = KFold(3)

scores = np.zeros(3)
for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(df_copy.iloc[:, :-1], df_copy.iloc[:, -1], test_size=0.33)
    df_train = pd.concat([X_train, y_train], axis=1)
    print("trial {}".format(i+1))
    model_train = model0 = sm.OLS.from_formula('dur1 ~ C(is_hol) + C(m_d) + C(dow) + C(hr) + C(pas_cnt)\
                            + scale(total_dist) + day_max_temp + day_min_temp + day_rainfall\
                            + day_snowfall + day_snowdepth + hr_temp + hr_humid + hr_dewpt\
                            + hr_windspeed + hr_gustspeed + hr_precip - 1', data=df_train)
    res_train = model_train.fit()
    y_pred = res_train.predict(X_test)
    y_pred[y_pred < 0] = 0
    try:
        scores[i] = mean_squared_log_error(y_test, y_pred)
        print(pd.concat([y_test, y_pred], axis=1).head(3))
    except:
        print("except")
        print(y_test[y_test<=0].shape)
        print(y_pred[y_pred<=0].shape)
        break

np.sqrt(scores)

trial 1
        dur1            0
536428  1292  1054.850803
568864   645   887.577978
130365  1193   751.683572
trial 2
        dur1            0
693795   903   772.144072
46071   1265  1287.358821
496313   475   376.483320
trial 3
        dur1            0
140487   603   814.823659
5861     751   696.027557
257313   666  1014.085731


array([ 0.48776671,  0.48784165,  0.4887244 ])

### to add: dow, m_d, hr, L2, holiday, disaster
## * dow

In [21]:
s_dow = df_train['pickup_datetime'].apply(pd.to_datetime)

In [23]:
s_time = s_dow.copy()

In [30]:
s_dow = s_time.dt.weekday_name

In [32]:
df_train.insert(4, 'dow', s_dow)

In [33]:
df_train.head()

,id,vendor_id,Time,pickup_datetime,dow,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,number_of_steps,street_for_each_step,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,Saturday,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,17,East 3rd Street|2nd Avenue|Chrystie Street|Del...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,Saturday,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,15,North 7th Street|Roebling Street|North 10th St...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,Saturday,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,13,8th Avenue|West 36th Street|1st Avenue|1st Ave...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,Saturday,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,6,Gramercy Park North|East 21st Street|Park Aven...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,Saturday,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,10,West 13th Street|Washington Street|West 14th S...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


In [34]:
df_copy = df_train

## * m_d

In [36]:
df_copy['pickup_datetime'] = df_copy['pickup_datetime'].apply(pd.to_datetime)

In [37]:
df_copy.insert(loc=4, column='m_d', value=np.vectorize(str)(df_copy['pickup_datetime'].dt.month*100
                                                             + df_copy['pickup_datetime'].dt.day))

In [38]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,m_d,dow,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,number_of_steps,street_for_each_step,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,17,East 3rd Street|2nd Avenue|Chrystie Street|Del...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,15,North 7th Street|Roebling Street|North 10th St...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,13,8th Avenue|West 36th Street|1st Avenue|1st Ave...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,6,Gramercy Park North|East 21st Street|Park Aven...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,10,West 13th Street|Washington Street|West 14th S...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * hr

In [39]:
df_copy.insert(loc=6, column='hr', value=df_copy['pickup_datetime'].dt.hour)

In [40]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,m_d,dow,hr,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,number_of_steps,...,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,17,...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,15,...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,13,...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,6,...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,10,...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * L2

In [41]:
def dist(plng, plat, dlng, dlat):
    return ((dlng-plng)**2 + (dlat-plat)**2) ** 0.5

In [42]:
df_copy.insert(loc=7, column='L2', value=np.vectorize(dist)(\
                df_copy['pickup_longitude'], df_copy['pickup_latitude'],\
                df_copy['dropoff_longitude'], df_copy['dropoff_latitude']))

df_copy.head()

,id,vendor_id,Time,pickup_datetime,m_d,dow,hr,L2,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,...,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.069424,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.057791,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.066698,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,0.018223,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,0.027198,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * holiday

In [51]:
holidays = {(2016, 1, 1) : 'New Year\'s',
            (2016, 1, 18) : 'MLK',
            (2016, 2, 15) : 'Washington',
            (2016, 5, 30) : 'Memorial' }

def is_holiday(timestamp):
    return (timestamp.year, timestamp.month, timestamp.day) in holidays

t = pd.Timestamp(2016, 5, 30)

print(is_holiday(t))

True


In [53]:
s_hol = df_copy['pickup_datetime'].apply(is_holiday)
s_hol.sum()

15583

In [64]:
df_copy.insert(loc=4, column='is_hol', value=s_hol)

In [67]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,is_hol,m_d,dow,hr,L2,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,...,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.069424,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.057791,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.066698,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.018223,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.027198,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * disaster

??

In [69]:
df_copy.to_csv('../_data/train_with__all_data_no_disaster_nans_fixed.csv', index=False)

---

In [70]:
pd.options.display.max_columns = 60

In [71]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,is_hol,m_d,dow,hr,L2,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,number_of_steps,street_for_each_step,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.069424,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,17,East 3rd Street|2nd Avenue|Chrystie Street|Del...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.057791,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,15,North 7th Street|Roebling Street|North 10th St...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.066698,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,13,8th Avenue|West 36th Street|1st Avenue|1st Ave...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.018223,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,6,Gramercy Park North|East 21st Street|Park Aven...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.027198,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,10,West 13th Street|Washington Street|West 14th S...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


In [72]:
droppable = ['id', 'vendor_id', 'Time', 'pickup_datetime', 'dropoff_datetime',\
             'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',\
             'store_and_fwd_flag', 'starting_street', 'end_street',\
             'number_of_steps', 'street_for_each_step', 'distance_per_step', 'travel_time_per_step',\
             'step_maneuvers', 'step_direction', 'step_location_list',\
             'average temperature', 'Wind Dir', 'Events', 'Conditions']

In [73]:
for col in droppable:
    del df_copy[col]

In [74]:
df_copy.head()

,is_hol,m_d,dow,hr,L2,passenger_count,trip_duration,total_distance,total_travel_time,maximum temperature,minimum temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Speed,Gust Speed,Precip
0,False,430,Saturday,23,0.069424,1,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [82]:
df_copy.to_csv('../_data/train__dropped__no_disaster.csv', index=False)

In [78]:
df_copy.columns

Index(['is_hol', 'm_d', 'dow', 'hr', 'L2', 'passenger_count', 'trip_duration',
       'total_distance', 'total_travel_time', 'maximum temperature',
       'minimum temperature', 'precipitation', 'snow fall', 'snow depth',
       'Temp.', 'Humidity', 'Dew Point', 'Wind Speed', 'Gust Speed', 'Precip'],
      dtype='object')

In [79]:
short_cols = ['is_hol', 'm_d', 'dow', 'hr', 'L2', 'pas_cnt', 'dur',
       'total_dist', 'total_time', 'day_max_temp',
       'day_min_temp', 'day_rainfall', 'day_snowfall', 'day_snowdepth',
       'hr_temp', 'hr_humid', 'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip']

In [80]:
df_copy.columns = short_cols

In [81]:
df_copy.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,dur,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [98]:
df_copy.isnull().sum()

is_hol              0
m_d                 0
dow                 0
hr                  0
L2                  0
pas_cnt             0
dur                 0
total_dist          0
total_time          0
day_max_temp        0
day_min_temp        0
day_rainfall        0
day_snowfall        0
day_snowdepth       0
hr_temp          5293
hr_humid         5293
hr_dewpt         5293
hr_windspeed     5293
hr_gustspeed     5293
hr_precip        5293
dtype: int64

In [226]:
nans = df_copy.loc[df_copy['hr_temp'].isnull(), ['m_d', 'hr']].copy()
nans['m_d'] = nans['m_d'].apply(int)
print(nans.head())
nansarray = np.array(nans)
np.unique(nansarray, axis=0)

       m_d  hr
67827  419  11
67828  419  11
67829  419  11
67830  419  11
67831  419  11


array([[313,   4],
       [105,   0],
       [105,   1],
       [105,   2],
       [105,   3],
       [108,  10],
       [401,   8],
       [401,   9],
       [401,  10],
       [401,  11],
       [401,  12],
       [401,  13],
       [401,  14],
       [402,   8],
       [403,   0],
       [403,   1],
       [404,   0],
       [404,   2],
       [409,   6],
       [413,  22],
       [413,  23],
       [416,   0],
       [419,  11],
       [202,  23]], dtype=int64)

In [221]:
nan_pairs = np.unique(nansarray, axis=0)

nan_pairs

array([[313,   4],
       [105,   0],
       [105,   1],
       [105,   2],
       [105,   3],
       [108,  10],
       [401,   8],
       [401,   9],
       [401,  10],
       [401,  11],
       [401,  12],
       [401,  13],
       [401,  14],
       [402,   8],
       [403,   0],
       [403,   1],
       [404,   0],
       [404,   2],
       [409,   6],
       [413,  22],
       [413,  23],
       [416,   0],
       [419,  11],
       [202,  23]], dtype=int64)

In [230]:
df_copy2 = df_copy.copy()

In [215]:
def get_prev(md, hr):
    m = int(md[0])
    d = int(md[1:])
    ts = pd.Timestamp(2016, m, d, hr, 0, 0)
    ts -= pd.Timedelta(hours=1)
    m_new = ts.month
    d_new = ts.day
    hr_new = ts.hour
    md_new = str(m_new)+'{:02}'.format(d_new)
    return(md_new, hr_new)

def get_next(m_d, hr):
    m = int(m_d[0])
    d = int(m_d[1:])
    ts = pd.Timestamp(2016, m, d, hr, 0, 0)
    ts += pd.Timedelta(hours=1)
    m_new = ts.month
    d_new = ts.day
    hr_new = ts.hour
    m_d_new = str(m_new)+'{:02}'.format(d_new)
    return(m_d_new, hr_new)

get_prev('105', 0)
get_next('630', 23)

('701', 0)

In [227]:
df_copy2['hr_temp'][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].iloc[0]

6.7000000000000002

In [231]:
fill_cols = ['hr_temp', 'hr_humid', 'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip']

for md, hr in nan_pairs:
    print(md, hr)
    md = str(md)
    md_prev, hr_prev = md, hr
    md_next, hr_next = md, hr
    while df_copy2[col][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].isnull().sum() > 0:
        md_prev, hr_prev = get_prev(md_prev, hr_prev)
    while df_copy2[col][(df_copy2['m_d'] == md_next) & (df_copy2['hr'] == hr_next)].isnull().sum() > 0:
        md_next, hr_next = get_next(md_next, hr_next)

    for col in fill_cols:
        print(col)
        col_prev = df_copy2[col][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].iloc[0]
        col_next = df_copy2[col][(df_copy2['m_d'] == md_next) & (df_copy2['hr'] == hr_next)].iloc[0]
        col_new = np.average([col_prev, col_next])
        print(col_new)
        df_copy2.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), col] = col_new
    print()


313 4
hr_temp
11.1
hr_humid
0.59
hr_dewpt
3.3
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

105 0
hr_temp
-9.7
hr_humid
0.515
hr_dewpt
-18.05
hr_windspeed
11.1
hr_gustspeed
33.3
hr_precip
0.0

105 1
hr_temp
-10.4
hr_humid
0.5375
hr_dewpt
-18.175
hr_windspeed
9.25
hr_gustspeed
33.3
hr_precip
0.0

105 2
hr_temp
-10.75
hr_humid
0.54875
hr_dewpt
-18.2375
hr_windspeed
8.325
hr_gustspeed
33.3
hr_precip
0.0

105 3
hr_temp
-10.925
hr_humid
0.554375
hr_dewpt
-18.26875
hr_windspeed
7.8625
hr_gustspeed
33.3
hr_precip
0.0

108 10
hr_temp
3.5
hr_humid
0.65
hr_dewpt
-2.5
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

401 8
hr_temp
20.0
hr_humid
0.68
hr_dewpt
13.85
hr_windspeed
4.65
hr_gustspeed
0.0
hr_precip
0.0

401 9
hr_temp
21.1
hr_humid
0.645
hr_dewpt
14.125
hr_windspeed
2.325
hr_gustspeed
0.0
hr_precip
0.0

401 10
hr_temp
21.65
hr_humid
0.6275
hr_dewpt
14.2625
hr_windspeed
1.1625
hr_gustspeed
0.0
hr_precip
0.0

401 11
hr_temp
21.925
hr_humid
0.61875
hr_dewpt
14.33125
hr_windspeed
0.58125
h

In [263]:
df_copy2.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,dur,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [265]:
df_copy2.to_csv('../_data/train__dropped__no_disaster_nan_fixed.csv', index=False)

In [264]:
df_copy3 = df_copy2.copy()
df_copy3.isnull().sum()

is_hol           0
m_d              0
dow              0
hr               0
L2               0
pas_cnt          0
dur              0
total_dist       0
total_time       0
day_max_temp     0
day_min_temp     0
day_rainfall     0
day_snowfall     0
day_snowdepth    0
hr_temp          0
hr_humid         0
hr_dewpt         0
hr_windspeed     0
hr_gustspeed     0
hr_precip        0
dtype: int64

In [236]:
md = '419'
hr = 11
df_copy2.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'].head()

67827    19.45
67828    19.45
67829    19.45
67830    19.45
67831    19.45
Name: hr_temp, dtype: float64

In [234]:
df_copy3.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'] = 3
df_copy3.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'].head()

67827    3.0
67828    3.0
67829    3.0
67830    3.0
67831    3.0
Name: hr_temp, dtype: float64

In [162]:
ssss[:]

67827   NaN
67828   NaN
67829   NaN
67830   NaN
67831   NaN
67832   NaN
67833   NaN
67834   NaN
67835   NaN
67836   NaN
67837   NaN
67838   NaN
67839   NaN
67840   NaN
67841   NaN
67842   NaN
67843   NaN
67844   NaN
67845   NaN
67846   NaN
67847   NaN
67848   NaN
67849   NaN
67850   NaN
67851   NaN
67852   NaN
67853   NaN
67854   NaN
67855   NaN
67856   NaN
         ..
68066   NaN
68067   NaN
68068   NaN
68069   NaN
68070   NaN
68071   NaN
68072   NaN
68073   NaN
68074   NaN
68075   NaN
68076   NaN
68077   NaN
68078   NaN
68079   NaN
68080   NaN
68081   NaN
68082   NaN
68083   NaN
68084   NaN
68085   NaN
68086   NaN
68087   NaN
68088   NaN
68089   NaN
68090   NaN
68091   NaN
68092   NaN
68093   NaN
68094   NaN
68095   NaN
Name: hr_temp, Length: 269, dtype: float64

In [ ]:
df_copy.to_csv('../_data/train__dropped__no_disaster.csv', index=False)

In [38]:
def make_hourly(timestamp):
    y = timestamp.year
    m = timestamp.month
    d = timestamp.day
    h = timestamp.hour
    return pd.Timestamp(y, m, d, h, 0, 0)

In [45]:
df_copy.insert(2, 'Time', df_copy['pickup_datetime'].apply(make_hourly))

In [44]:
del df_copy['pickup_hour']

In [46]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,...,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,...,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,...,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0


In [47]:
df_copy.to_csv('../_data/train_with_osrm_and_weather_h.csv')

In [42]:
df_hourly.head()

,Time,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,2015-12-31 02:00:00,7.8,0.89,6.1,NNE,5.6,0.0,0.8,None,Overcast
1,2015-12-31 03:00:00,7.2,0.90,5.6,Variable,7.4,0.0,0.3,None,Overcast
2,2015-12-31 04:00:00,7.2,0.90,5.6,Calm,0.0,0.0,0.0,None,Overcast
3,2015-12-31 05:00:00,7.2,0.86,5.0,NW,7.4,0.0,0.0,None,Overcast
4,2015-12-31 06:00:00,7.2,0.90,5.6,West,5.6,0.0,0.0,None,Overcast


In [48]:
df_merged = df_copy.merge(df_hourly, how='left', on='Time')

In [64]:
df_hourly.iloc[2901]

Time          2016-04-30 23:00:00
Temp.                        10.6
Humidity                     0.56
Dew Point                     2.2
Wind Dir                      SSE
Wind Speed                     13
Gust Speed                      0
Precip                          0
Events                       None
Conditions                  Clear
Name: 2901, dtype: object

In [49]:
df_merged.head()

,id,vendor_id,Time,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


### df + osrm + weather + hourly weather 저장

In [65]:
df_merged.to_csv('../_data/train_with_osrm_and_weather_hourly.csv')